In [19]:
### Importing ###
from numpy import exp, array, random, dot #imports linear algebra operations/types
import numpy as np #operations in numpy
from scipy import misc #misc. operations in scipy
import skimage
import math 
from __future__ import with_statement

In [20]:
### Get picture data ###
arr1 = np.divide(misc.imread('save_save_save_america02.png'),255) #read 1st image, divide by 255 for number between 0 and 1
arr1r = arr1[:,:,0] #reds
arr1g = arr1[:,:,1] #greens
arr1b = arr1[:,:,2] #blues
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [21]:
### Hyper-parameters ###
layerSize = 3 #number of filters in a layer
layerNum = 6 #number of layers
filterSize = 10 #size of filter
epochs = 10 #number of times trained
filters = [] #init filter
bias = 0.01
targets = np.ones((len(arr1r)*len(arr1r[0])*layerSize)) #target values

In [22]:
### Variables based on hyper-params ###
filterNum = layerSize**2 #number of filters per layer
totalFilterNum = filterNum*layerNum #total number of filters

In [23]:
### Set filters ###
def setFilters():
    global filters #Filters is a global variable
    filters = 0.01*random.random((layerNum, filterNum, filterSize, filterSize))-0.005 #Set filters to random 4d array
setFilters()
print(filters)

[[[[ -2.97503212e-03  -1.18400263e-03  -3.19878318e-03 ...,
     -1.71962154e-03   3.09708428e-03   1.32055781e-03]
   [  2.27452242e-03   4.64845135e-03   3.13510377e-03 ...,
      2.75874157e-04  -4.95095900e-04  -3.86889524e-03]
   [  2.82698253e-04   3.14475939e-03   4.48648830e-03 ...,
      7.85265919e-04   1.77927698e-03  -1.51387361e-03]
   ..., 
   [  1.12089997e-03   7.56549248e-04   1.67277674e-03 ...,
     -2.75174012e-03   1.61309563e-03  -5.29587530e-06]
   [  4.35643538e-03   9.67834295e-04   2.71358831e-03 ...,
      2.92106923e-03  -2.82116478e-03   1.28804263e-03]
   [  2.45295122e-03   2.89851774e-03  -2.10086756e-03 ...,
      1.15197053e-03  -4.74037900e-03   4.85134609e-03]]

  [[ -8.61885513e-06  -8.89151904e-04   4.52798783e-03 ...,
     -1.33630974e-03  -1.03885854e-03  -1.32199001e-03]
   [ -1.23291214e-03  -1.13140121e-04  -3.62185188e-03 ...,
     -1.89388627e-03  -8.90335599e-04   2.82973375e-03]
   [  4.06798720e-03   1.06378823e-03  -1.18328578e-03 ...,
 

In [98]:
### Helper functions ###
#Activation function (Leaky ReLU)
def activation(x, deriv=False, leak=-0.01):
    new_x=np.copy(array(x)) #Create a copy of x
    if deriv: #If you want to activate derivative
        #Calculation of derivative of leaky ReLU
        new_x[x > 0] = 1 
        new_x[x <= 0] = leak
        return new_x
    #Calculation of leaky ReLU
    new_x[x < 0] *= leak
    new_x = x * (x > 0)
    return new_x
#Sum with no negatives (not used)
'''
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
'''
#Apply cross-correlation for 1 index
def apply(filterArr, img, row, col):
    value = 0 #Init value
    halfFil = int(filterSize/2) #Half of filterSize
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0)) #Pad image with 0's
    for x in range(filterSize): 
        for y in range(filterSize): #Apply over all filter
            value += padImg[x+row][y+col]*filterArr[x][y] #Add image multiplied by filter
    return value
#Softmax function
def softmax(x):
#     exps = np.exp(x - x.max())
#     return exps / np.sum(exps)
    return np.exp(x) / np.sum(np.exp(x), axis=0) #Calculate softmax
print(apply(filters[1][1],arr1r,0,0))

-138.248043855


In [30]:
### Apply filters ###
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])): #Apply over all image
            new_img[y][x] += activation(apply(filterArr,img,y,x))+bias#Add up all cross-correlations (with ReLU)
    return new_img

In [31]:
### Pooling ###
def pool(img, size):
    pad_w = size-len(img)%size
    pad_h = size-len(img[0])%size
    if pad_w == size:
        pad_w = 0
    if pad_h == size:
        pad_h = 0
    new_img = np.lib.pad(array(img), ((0,pad_w),(0,pad_h)), 'edge') #Pad image
    k = len(new_img)/size #Width of new image 
    l = len(new_img[0])/size #Height of new image
    return new_img.reshape(int(k),size,int(l),size).max(axis=(-1,-3)) #Arrange image into groups of size*size, then get max on axis

In [32]:
### Forward-propagation ###
def forward(img):
    outputLayers = [[img[:,:,0],img[:,:,1],img[:,:,2]]] #Get all the image 
    for num in range(layerNum):
        outputLayers.append([]) #Append a new layer
        for lay in range(layerSize):
            outputLayers[num+1].append(np.zeros((len(outputLayers[num][lay]), len(outputLayers[num][lay][0]))))
            # Add components of the layer
            outputLayers[num][lay] = pool(outputLayers[num][lay], 2)
            # Do pooling
        #lay1 is the layer index you are at and lay2 is the layer index you are going to
        for lay1 in range(layerSize):
            for lay2 in range(layerSize):
                newImage = outputLayers[num+1][lay2] #Get refrence to next layer
#                 newImage += bias
                #print("S-O: ("+str(len(outputLayers))+", "+str(len(outputLayers[0]))+")")
                #print("S-F: ("+str(len(filters))+", "+str(len(filters[0]))+")")
                applyFilters(outputLayers[num][lay1], newImage, filters[num][layerSize*lay1+lay2])
                #Apply corresponding filter
                if (filterNum*num + layerSize*lay1+lay2) % 20 == 0:
                    print(filterNum*num + layerSize*lay1+lay2)
#                 print(newImage)
    outputLayers.append(softmax(array(outputLayers[len(outputLayers)-1]).flatten())) #Add flattened softmaxed array
    return outputLayers
out = forward(arr1)
print(out)

0
20
40
[[array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.23529412,  1.        ,  0.23529412,  1.        ,  0.23529412,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [ 1.        ,  0.23529412,  0.23529412,  1.        ,  0.23529412,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [ 0.23529412,  1.        ,  0.23529412,  1.        ,  0.35686275,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [ 0.92941176,  0.92941176,  0.92941176,  0.92941176,  0.92941176,
         0.92941176,  0.92941176,  0.92941176,  0.92941176,  0.92941176,
         0.92941176,  0.92941176,  0.92941176],
       [ 1.        ,

In [99]:
### Error + Back-propagation ###
error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets)) # negative sum of log(out) times target- cross-entropy
print(error)
def back(layer, grad):
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize))
    for x in range(filterNum):
        r = 0
#         print("Filter S:"+str(len(filters)))
#         print(layerNum)
        for row in filters[layer-1][x]:
            c=0
            for w in row:
                new_grad[x][r][c]=-np.multiply(error,np.mean(np.multiply(np.multiply(
                    activation(array(out[layer-1]),deriv=True),
                    w),
                    grad[x][r][c])))
#                 print(-np.divide(error,np.sum(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]))))
#                 print(new_grad[x][r][c])
                filters[layer-1][x][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

FloatingPointError: divide by zero encountered in log10

In [106]:
setFilters()
np.seterr(divide='raise')
np.seterr(invalid='raise')
np.seterr(over='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'raise', 'under': 'ignore'}

In [107]:
for x in range(epochs):
    out = forward(arr1)
    error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets))
    print(error)
    print(filters)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#     print(filters)

0
20
40
3695.06411553
[[[[ -4.20446999e-03  -1.65124238e-03  -4.18788292e-03 ...,
     -2.65493344e-03  -2.25901730e-04   4.17021395e-03]
   [  3.97871375e-03  -1.60549029e-03  -3.06648869e-03 ...,
      3.52229634e-03   4.15582659e-03   1.63437630e-03]
   [  1.54900911e-04   3.83223473e-03   4.17280007e-03 ...,
      2.42951618e-03  -3.12109386e-03  -3.93752732e-03]
   ..., 
   [  1.46578233e-03   8.86811252e-04  -4.07674896e-03 ...,
      1.02824071e-03  -3.79279063e-04   4.02553983e-03]
   [ -3.46921472e-03   2.16611283e-03  -4.47638763e-03 ...,
     -3.78858625e-03  -1.70845947e-03  -4.97729507e-03]
   [ -2.01235211e-03   2.07341584e-03   3.55166160e-03 ...,
      3.13135906e-03   1.01850542e-03   4.57254920e-04]]

  [[  1.41017634e-03  -4.55350839e-03  -1.37751884e-03 ...,
     -4.89211301e-03  -6.41506452e-04  -2.95654695e-03]
   [ -2.26385291e-03   1.08328050e-03  -7.31280057e-04 ...,
      4.75653451e-03   1.85833405e-03   3.35315766e-05]
   [ -1.93321106e-03  -2.66990034e-03  

FloatingPointError: overflow encountered in exp